In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from indexing_newv2 import MeetingsScanner, ActiveMeetingsMonitor, VexaAPI, VexaAuth, IndexingWorker,RedisMonitor,RedisKeys,IndexingPipeline

In [11]:
from redis import Redis

In [12]:
import os

In [13]:
redis = Redis(
            host=os.getenv('REDIS_HOST', '127.0.0.1'),
            port=int(os.getenv('REDIS_PORT', 6379)),
            db=0,
            decode_responses=True,
            max_connections=10,  # Add connection pooling
            socket_timeout=5
        )

In [14]:
pipeline = IndexingPipeline()
await pipeline.start()


/home/dima/anaconda3/envs/langchain/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

Vexa token: 5ce9e6757a7f4ec18ff87c5001787a3c
User information retrieved successfully.
Vexa token: 5ce9e6757a7f4ec18ff87c5001787a3c
Request URL: http://127.0.0.1:8001/api/v1/calls/all
Request Params: {'token': '5ce9e6757a7f4ec18ff87c5001787a3c'}
Vexa token: e79a7431614546f3b1ec5930ff91015e
User information retrieved successfully.
Vexa token: e79a7431614546f3b1ec5930ff91015e
Request URL: http://127.0.0.1:8001/api/v1/calls/all
Request Params: {'token': 'e79a7431614546f3b1ec5930ff91015e'}
Vexa token: 5faa86ba0dc74b14a78c60c8aa20a014
User information retrieved successfully.
Vexa token: 5faa86ba0dc74b14a78c60c8aa20a014
Request URL: http://127.0.0.1:8001/api/v1/calls/all
Request Params: {'token': '5faa86ba0dc74b14a78c60c8aa20a014'}
Vexa token: 5ce9e6757a7f4ec18ff87c5001787a3c
Request URL: http://127.0.0.1:8001/api/v1/calls/all
Request Params: {'token': '5ce9e6757a7f4ec18ff87c5001787a3c'}
Vexa token: e79a7431614546f3b1ec5930ff91015e
Request URL: http://127.0.0.1:8001/api/v1/calls/all
Request P

In [6]:
redis_stats = RedisMonitor(redis)


In [7]:
#redis_stats.reset_pipeline()

In [8]:
monitor = ActiveMeetingsMonitor(redis, VexaAuth())
await monitor._collect_active_users()

In [9]:
redis_stats.get_pipeline_stats()

{'users': {'count': 3,
  'items': ['638129fd-9722-4430-bd1d-cd3f9a0c3dd6',
   'd47466b3-fb76-42ea-8ed2-0d05f1a64486',
   '8b767b85-44fb-4c20-97df-7aa172242273']},
 'meetings': {'queued': {'count': 0, 'items': []},
  'processing': {'count': 0, 'items': []},
  'failed': {'count': 0, 'items': {}},
  'retries': {}}}

In [10]:

self = MeetingsScanner(redis, VexaAuth())


In [11]:
seen_users = self.redis.smembers(RedisKeys.SEEN_USERS)

In [12]:
seen_users

{'638129fd-9722-4430-bd1d-cd3f9a0c3dd6',
 '8b767b85-44fb-4c20-97df-7aa172242273',
 'd47466b3-fb76-42ea-8ed2-0d05f1a64486'}

In [13]:
user_id = list(seen_users)[0]
await self._scan_user_meetings(user_id)


Vexa token: e79a7431614546f3b1ec5930ff91015e
Request URL: http://127.0.0.1:8001/api/v1/calls/all
Request Params: {'token': 'e79a7431614546f3b1ec5930ff91015e'}


In [14]:
redis_stats.get_pipeline_stats()

{'users': {'count': 3,
  'items': ['638129fd-9722-4430-bd1d-cd3f9a0c3dd6',
   'd47466b3-fb76-42ea-8ed2-0d05f1a64486',
   '8b767b85-44fb-4c20-97df-7aa172242273']},
 'meetings': {'queued': {'count': 3,
   'items': [{'id': '6fb41ae1-e7ae-4005-9d00-6e6f665bb674',
     'score': 1731346871.789839},
    {'id': '8d0f7a03-3f3f-4f1e-b1db-3bc9cbbc400b', 'score': 1731346871.790121},
    {'id': '6e7334f3-544b-4b0a-a787-cf7299d45579',
     'score': 1731346871.790327}]},
  'processing': {'count': 0, 'items': []},
  'failed': {'count': 0, 'items': {}},
  'retries': {}}}

In [15]:
self = IndexingWorker(redis)

/home/dima/anaconda3/envs/langchain/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

In [16]:
next_meetings = self.redis.zrevrange(RedisKeys.INDEXING_QUEUE, 0, 0, withscores=True)

In [17]:
next_meetings

[('6e7334f3-544b-4b0a-a787-cf7299d45579', 1731346871.790327)]

In [18]:
meeting_id, score = next_meetings[0]

In [19]:
self.redis.sismember(RedisKeys.PROCESSING_SET, meeting_id)

0

In [25]:
await self._process_meeting(meeting_id)

Vexa token: e79a7431614546f3b1ec5930ff91015e
User information retrieved successfully.
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3
Model device: cuda:3


In [26]:
redis_stats.get_pipeline_stats()

{'users': {'count': 3,
  'items': ['638129fd-9722-4430-bd1d-cd3f9a0c3dd6',
   'd47466b3-fb76-42ea-8ed2-0d05f1a64486',
   '8b767b85-44fb-4c20-97df-7aa172242273']},
 'meetings': {'queued': {'count': 2,
   'items': [{'id': '6fb41ae1-e7ae-4005-9d00-6e6f665bb674',
     'score': 1731346871.789839},
    {'id': '8d0f7a03-3f3f-4f1e-b1db-3bc9cbbc400b',
     'score': 1731346871.790121}]},
  'processing': {'count': 0, 'items': []},
  'failed': {'count': 0, 'items': {}},
  'retries': {}}}

In [49]:
redis_stats.reset_pipeline()

{'status': 'reset', 'keys_deleted': 4}